In [1]:
from tensorflow import keras
import tensorflow as tf
import os

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
import numpy as np

from sklearn.preprocessing import StandardScaler

In [2]:
root_logdir = os.path.join(os.curdir, "my_logs")

In [3]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [4]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [5]:
X_train_full.shape

(15480, 8)

In [6]:
np.random.seed(42)
tf.random.set_seed(42)

In [7]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [8]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [9]:
from numpy import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np


param_distribs = {
    "n_hidden" : range(4),
    "n_neurons" : np.arange(1, 100).tolist(),
    "learning_rate" : np.arange(3e-4, 3e-2).tolist()
    
}
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv = 3, verbose=1, n_jobs = 1)
rnd_search_cv.fit(X_train, y_train, 
                  epochs = 100, 
                  validation_data = (X_valid, y_valid), 
                  callbacks = keras.callbacks.EarlyStopping(patience=10), verbose = 1)



Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/100
242/242 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 4/100
242/242 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 5/100
242/242 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 7/100
242/242 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 9/100
242/242 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 10/100
121/121 [==============================] - 0s 798us/step - loss: nan
Epoch 1/100
242/242 [==============================] - 0s 2ms/s

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  2.2min finished


RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x0000020C1627BA48>, as the constructor either does not set or modifies parameter learning_rate

In [ ]:
rnd_search_cv.best_params_

The best parameters are {'n_neurons': 59, 'n_hidden': 2, 'learning_rate': 0.03}

In [ ]:
rnd_search_cv.best_score_

In [ ]:
model = rnd_search_cv.best_estimator_.model

import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize = (8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

model.evaluate(X_test, y_test)

X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

import numpy as np

y_pred = np.argmax(model.predict(X_new), axis=-1)
y_pred

np.array(class_names)[y_pred]

y_test[:3]